# Graphistry Plots

In [2]:
NEO4J = {
    'uri': "bolt://matlaber5.media.mit.edu:7687", 
    'auth': ("neo4j", "myneo")
}

# !pip install pandas
# !pip install graphistry[bolt] --user

import pandas as pd
import graphistry
import time

graphistry.register(key='3bd0ff5a5304a3ee27de2ca78ac7b67bbc48dc67409c255c0f70250955cf967c')
graphistry.register(bolt=NEO4J)
graphistry.__version__

'0.9.64'

# Individual Coauthorship Communities

In [3]:
g = graphistry.cypher("""
MATCH p=(a:Author)-[:COAUTHOR]-(:Author)-[:COAUTHOR]-(:Author)
WHERE a.name = "Raja Srinivas"
RETURN p
  """, 
  {})
g = g.bind(edge_weight='strength', point_label='name') 
g.plot()

# Journal Coauthorship Communities

In [4]:
top_5 = ['Cell', 'Nature', 'Nature Biotechnology',
            'Proceedings of the National Academy of Sciences of the United States of America',
            'Science']
 
top_10 = ['Cell', 'Nature', 'Nature Biotechnology',
           'Proceedings of the National Academy of Sciences of the United States of America',
           'Science', 'Journal of the American Chemical Society', 'JAMA', 
           'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
 
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research',
          'Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology',
          'The EMBO Journal','Genes & Development','Immunity','Journal of Neurology',
          'Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry',
          'Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine',
          'Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology',
          'Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology',
          'The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology',
          'Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics',
          'PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America',
          'Science Signaling','Science Translational Medicine','Science']

In [6]:
venues = top_5
min_pubs = 50
min_cowritten = 5

g = graphistry.cypher("""
MATCH (a:Author)-[r:AUTHORED]->(q:Quanta)
WHERE q.venue in {}
WITH a, count(q) as num_publications
WHERE num_publications > {}
WITH apoc.coll.toSet(COLLECT(a)) AS author_list
MATCH p=(m:Author)-[c:COAUTHOR]->(n:Author)
WHERE (m IN author_list) AND (n IN author_list) AND (m.name>n.name) AND (c.strength42 > {})
RETURN p""".format(venues, min_pubs, min_cowritten), {})

g = g.bind(edge_weight='strength42', point_label='name', edge_color='strength42') 
g.plot()